In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from catboost import CatBoostClassifier
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
PATH_TO_DATA = ''

import ujson as json 
# Any results you write to the current directory are saved as output.

In [3]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                             'train_features.csv.zip'), 
                                    index_col='match_id_hash')
train_targets = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                            'train_targets.csv.zip'), 
                                   index_col='match_id_hash')
train_df.head()

,game_time,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,...,d5_stuns,d5_creeps_stacked,d5_camps_stacked,d5_rune_pickups,d5_firstblood_claimed,d5_teamfight_participation,d5_towers_killed,d5_roshans_killed,d5_obs_placed,d5_sen_placed
match_id_hash,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,155,22,7,1,11,11,0,0,0,0,...,0.000000,0,0,0,0,0.00,0,0,0,0
b9c57c450ce74a2af79c9ce96fac144d,658,4,0,3,10,15,7,2,0,7,...,0.000000,0,0,0,0,0.00,0,0,0,0
6db558535151ea18ca70a6892197db41,21,23,0,0,0,101,0,0,0,0,...,0.000000,0,0,0,0,0.00,0,0,0,0
46a0ddce8f7ed2a8d9bd5edcbb925682,576,22,7,1,4,14,1,0,3,1,...,8.664527,3,1,3,0,0.00,0,0,2,0
b1b35ff97723d9b7ade1c9c3cf48f770,453,22,7,1,3,42,0,1,1,0,...,0.000000,2,1,2,0,0.25,0,0,0,0


In [4]:
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                             'test_features.csv.zip'), 
                                    index_col='match_id_hash')

test_df.head()

,game_time,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,...,d5_stuns,d5_creeps_stacked,d5_camps_stacked,d5_rune_pickups,d5_firstblood_claimed,d5_teamfight_participation,d5_towers_killed,d5_roshans_killed,d5_obs_placed,d5_sen_placed
match_id_hash,,,,,,,,,,,,,,,,,,,,,
30cc2d778dca82f2edb568ce9b585caa,23,4,0,0,0,79,0,0,0,0,...,0.000000,0,0,1,0,0.000000,0,0,0,0
70e5ba30f367cea48793b9003fab9d38,1044,22,7,12,6,23,3,5,7,1,...,67.025820,2,1,4,0,0.833333,0,0,5,2
4d9ef74d3a2025d79e9423105fd73d41,1091,22,7,6,1,64,3,1,7,1,...,0.000000,0,0,1,0,0.384615,1,0,1,0
2bb79e0c1eaac1608e5a09c8e0c6a555,623,22,7,1,0,41,0,0,1,2,...,9.564303,0,0,3,0,1.000000,0,0,0,0
bec17f099b01d67edc82dfb5ce735a43,1538,22,7,7,11,40,2,7,12,0,...,16.607927,0,0,2,1,0.590909,0,0,0,0


In [5]:
train_targets.radiant_win = train_targets.radiant_win.map({False: 0, True: 1})
y_train = train_targets.radiant_win

In [1]:
a = test_df.columns[5:30]
b = []
for i in a:
    b.append(i[3:])
key = []
for i in data_json['players'][9].keys():
    if i not in b:
        key.append(i)

NameError: name 'test_df' is not defined

In [9]:
key

['player_slot',
 'hero_name',
 'account_id_hash',
 'ability_upgrades',
 'observers_placed',
 'max_hero_hit',
 'times',
 'gold_t',
 'lh_t',
 'dn_t',
 'xp_t',
 'obs_log',
 'sen_log',
 'obs_left_log',
 'sen_left_log',
 'purchase_log',
 'kills_log',
 'buyback_log',
 'runes_log',
 'obs',
 'sen',
 'actions',
 'pings',
 'purchase',
 'gold_reasons',
 'xp_reasons',
 'killed',
 'item_uses',
 'ability_uses',
 'hero_hits',
 'damage',
 'damage_taken',
 'damage_inflictor',
 'runes',
 'killed_by',
 'kill_streaks',
 'multi_kills',
 'life_state',
 'healing',
 'damage_inflictor_received',
 'randomed',
 'pred_vict',
 'hero_inventory',
 'hero_stash',
 'nearby_creep_death_count']

In [8]:
with open(os.path.join(PATH_TO_DATA, 'train_matches.jsonl')) as fin:
        for i in range(1):
            first_line = fin.readline()
            data_json = json.loads(first_line)
data_json.keys()

dict_keys(['game_time', 'match_id_hash', 'teamfights', 'objectives', 'chat', 'game_mode', 'lobby_type', 'players', 'targets'])

In [18]:
train_df.columns[5:30]

Index(['r1_hero_id', 'r1_kills', 'r1_deaths', 'r1_assists', 'r1_denies',
       'r1_gold', 'r1_lh', 'r1_xp', 'r1_health', 'r1_max_health',
       'r1_max_mana', 'r1_level', 'r1_x', 'r1_y', 'r1_stuns',
       'r1_creeps_stacked', 'r1_camps_stacked', 'r1_rune_pickups',
       'r1_firstblood_claimed', 'r1_teamfight_participation',
       'r1_towers_killed', 'r1_roshans_killed', 'r1_obs_placed',
       'r1_sen_placed', 'r2_hero_id'],
      dtype='object')

In [11]:
data_json['players']

[{'player_slot': 0,
  'hero_id': 11,
  'hero_name': 'npc_dota_hero_nevermore',
  'account_id_hash': '86bf86c5488742478688149155258ea8',
  'ability_upgrades': [],
  'obs_placed': 0,
  'sen_placed': 0,
  'creeps_stacked': 0,
  'camps_stacked': 0,
  'rune_pickups': 1,
  'firstblood_claimed': 0,
  'teamfight_participation': 0,
  'towers_killed': 0,
  'roshans_killed': 0,
  'observers_placed': 0,
  'stuns': 0,
  'max_hero_hit': {'type': 'max_hero_hit',
   'time': 94.977,
   'max': True,
   'inflictor': 'nevermore_shadowraze2',
   'unit': 'npc_dota_hero_nevermore',
   'key': 'npc_dota_hero_tinker',
   'value': 136,
   'slot': 0,
   'player_slot': 0},
  'times': [0, 60, 120],
  'gold_t': [1, 91, 356],
  'lh_t': [0, 0, 4],
  'dn_t': [0, 0, 0],
  'xp_t': [0, 107, 387],
  'obs_log': [],
  'sen_log': [],
  'obs_left_log': [],
  'sen_left_log': [],
  'purchase_log': [{'time': -89.783, 'key': 'wraith_band'},
   {'time': -89.783, 'key': 'enchanted_mango'},
   {'time': -89.783, 'key': 'tpscroll'},
  

In [12]:
train_df.r1_gold.describe()

count    39675.000000
mean      7193.676673
std       6507.500421
min          2.000000
25%       2201.000000
50%       5422.000000
75%      10368.500000
max      58437.000000
Name: r1_gold, dtype: float64

In [91]:
def extract_features_csv_mod(match):
    row = [
        ('match_id_hash', match['match_id_hash']),
    ]
        
    for slot, player in enumerate(match['players']):
        if slot < 5:
            player_name = 'r%d' % (slot + 1)
        else:
            player_name = 'd%d' % (slot - 4)
        try:
            row.append(('%s_ability_level' % (player_name), (player['ability_upgrades'][-1]['level'])))
            row.append(('%s_max_ability' % (player_name), (player['ability_upgrades'][-1]['ability'])))
            row.append(('%s_ability_rate' % (player_name), (player['ability_upgrades'][-1]['ability']/player['ability_upgrades'][-1]['time'])))
        except:
            row.append(('%s_ability_level' % (player_name), 0))
            row.append(('%s_max_ability' % (player_name),0))
            row.append(('%s_ability_rate' % (player_name), 0))
        try:
            row.append(('%s_max_hero_hit_value' % (player_name), (player['max_hero_hit']['value'])))
        except:
            row.append(('%s_max_hero_hit_value' % (player_name), 0))
        try:
            row.append(('%s_gold_rate' % (player_name), (player['gold_t'][-1]/player['times'][-1])))
            row.append(('%s_xp_rate' % (player_name), (player['xp_t'][-1]/player['times'][-1])))
            row.append(('%s_dn_rate' % (player_name), (player['dn_t'][-1]/player['times'][-1])))
            row.append(('%s_lh_rate' % (player_name), (player['lh_t'][-1]/player['times'][-1])))
        except:
            row.append(('%s_gold_rate' % (player_name),0))
            row.append(('%s_xp_rate' % (player_name), 0))
            row.append(('%s_dn_rate' % (player_name), 0))
            row.append(('%s_lh_rate' % (player_name), 0))
        try:
            row.append(('%s_item_uses' % (player_name), (sum(player['item_uses'].values()))))
            row.append(('%s_item_uses_rate' % (player_name), (sum(player['item_uses'].values()))))
        except:
            row.append(('%s_item_uses' % (player_name), 0))
        try:
            row.append(('%s_ability_uses' % (player_name), (sum(player['ability_uses'].values()))))
        except:
            row.append(('%s_ability_uses' % (player_name), 0))
        try:
            row.append(('%s_damage_total' % (player_name), (sum(player['damage'].values()))))
        except:
            row.append(('%s_damage_total' % (player_name), 0))
        try:
            row.append(('%s_damage_taken_total' % (player_name), (sum(player['damage_taken'].values()))))
        except:
            row.append(('%s_damage_taken_total' % (player_name), 0))
        try:
            row.append(('%s_damage_inf_rec_total' % (player_name), (sum(player['damage_inflictor_received'].values()))))
        except:
            row.append(('%s_damage_inf_rec_total' % (player_name), 0))
        try:
            row.append(('%s_killed_total' % (player_name), (sum(player['killed'].values()))))
        except:
            row.append(('%s_killed_total' % (player_name), 0))

    return collections.OrderedDict(row)

In [23]:
import os

try:
    import ujson as json
except ModuleNotFoundError:
    import json
    print ('Please install ujson to read JSON oblects faster')
    
try:
    from tqdm import tqdm_notebook
except ModuleNotFoundError:
    tqdm_notebook = lambda x: x
    print ('Please install tqdm to track progress with Python loops')

def read_matches(matches_file):
    
    MATCHES_COUNT = {
        'test_matches.jsonl': 10000,
        'train_matches.jsonl': 39675,
    }
    _, filename = os.path.split(matches_file)
    total_matches = MATCHES_COUNT.get(filename)
    
    with open(matches_file) as fin:
        for line in tqdm_notebook(fin, total=total_matches):
            yield json.loads(line)

In [230]:
def add_new_features(df_features, matches_file):
    
    # Process raw data and add new features
    for match in read_matches(matches_file):
        match_id_hash = match['match_id_hash']

        # Counting ruined towers for both teams
        radiant_tower_kills = 0
        dire_tower_kills = 0
        for objective in match['objectives']:
            if objective['type'] == 'CHAT_MESSAGE_TOWER_KILL':
                if objective['team'] == 2:
                    radiant_tower_kills += 1
                if objective['team'] == 3:
                    dire_tower_kills += 1

        # Write new features
        df_features.loc[match_id_hash, 'radiant_tower_kills'] = radiant_tower_kills
        df_features.loc[match_id_hash, 'dire_tower_kills'] = dire_tower_kills
        df_features.loc[match_id_hash, 'diff_tower_kills'] = radiant_tower_kills - dire_tower_kills
        

In [32]:
import collections


PLAYER_FIELDS = [
 'ability_upgrades',
 'observers_placed',
 'max_hero_hit',
 'times',
 'obs_log',
 'sen_log',
 'obs_left_log',
 'sen_left_log',
 'purchase_log',
 'buyback_log',
 'runes_log',
 'actions',
 'pings',
 'gold_reasons',
 'xp_reasons',
 'killed',
 'item_uses',
 'ability_uses',
 'hero_hits',
 'damage',
 'damage_taken',
 'damage_inflictor',
 'runes',
 'multi_kills',
 'life_state',
 'healing',
 'damage_inflictor_received',
 'hero_inventory',
 'hero_stash',
 'nearby_creep_death_count'
]

def extract_features_csv(match):
    row = [
        ('match_id_hash', match['match_id_hash']),
    ]
        
    for slot, player in enumerate(match['players']):
        if slot < 5:
            player_name = 'r%d' % (slot + 1)
        else:
            player_name = 'd%d' % (slot - 4)

        for field in PLAYER_FIELDS:
            if (type(player[field]) == int) or (type(player[field]) == float) or (type(player[field]) == bool):
                column_name = '%s_%s' % (player_name, field)
                row.append((column_name, player[field]))
            else:
                column_name = '%s_%s' % (player_name, field)
                row.append((column_name, len(player[field])))
    return collections.OrderedDict(row)
    

In [92]:
%%time

df_new_features = []
for match in read_matches(os.path.join(PATH_TO_DATA, 'train_matches.jsonl')):
    match_id_hash = match['match_id_hash']
    features = extract_features_csv_mod(match)

    df_new_features.append(features)



Wall time: 1min 30s


In [93]:
df_new_features = pd.DataFrame.from_records(df_new_features).set_index('match_id_hash')
df_new_features.head(5).T

,r1_ability_level,r1_max_ability,r1_ability_rate,r1_max_hero_hit_value,r1_gold_rate,r1_xp_rate,r1_dn_rate,r1_lh_rate,r1_item_uses,r1_item_uses_rate,...,d5_xp_rate,d5_dn_rate,d5_lh_rate,d5_item_uses,d5_item_uses_rate,d5_ability_uses,d5_damage_total,d5_damage_taken_total,d5_damage_inf_rec_total,d5_killed_total
match_id_hash,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,0,0,0.000000,136,2.966667,3.225000,0.000000,0.033333,2,2,...,5.133333,0.000000,0.066667,3,3,4,2332,681,459,663
b9c57c450ce74a2af79c9ce96fac144d,6,5085,8.322422,216,8.281667,6.561667,0.011667,0.086667,34,34,...,1.893333,0.001667,0.013333,9,9,7,2308,2154,1403,462
6db558535151ea18ca70a6892197db41,0,0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0,0,...,0.000000,0.000000,0.000000,1,1,0,0,0,0,0
46a0ddce8f7ed2a8d9bd5edcbb925682,2,5075,11.353468,182,2.887037,2.427778,0.001852,0.000000,10,10,...,3.270370,0.001852,0.003704,11,11,19,3955,3317,2548,155
b1b35ff97723d9b7ade1c9c3cf48f770,2,5087,11.857809,60,3.226190,3.083333,0.000000,0.021429,5,5,...,4.173810,0.019048,0.066667,6,6,9,10739,2785,1432,2963
19c39fe2af2b547e48708ca005c6ae74,0,0,0.000000,90,2.850000,1.908333,0.008333,0.000000,1,1,...,1.166667,0.000000,0.050000,2,2,2,4496,948,0,538
c96d629dc0c39f0c616d1949938a6ba6,4,6342,8.068702,178,2.866667,2.586458,0.005208,0.001042,24,24,...,3.313542,0.008333,0.004167,14,14,32,4682,1608,1158,973
156c88bff4e9c4668b0f53df3d870f1b,4,5065,9.834951,110,6.024074,6.368519,0.016667,0.068519,27,27,...,1.268519,0.000000,0.003704,8,8,19,2156,1593,996,154
df50f738c454d430a33b2bf90cb9ef1a,5,5372,8.486572,295,4.748333,6.698333,0.005000,0.053333,5,5,...,8.853333,0.026667,0.110000,9,9,92,31098,3388,1929,11786


In [95]:
%%time

df_new_features_test = []
for match in read_matches(os.path.join(PATH_TO_DATA, 'test_matches.jsonl')):
    match_id_hash = match['match_id_hash']
    features = extract_features_csv_mod(match)

    df_new_features_test.append(features)
df_new_features_test = pd.DataFrame.from_records(df_new_features_test).set_index('match_id_hash')
df_new_features_test.head()


Wall time: 24.8 s


In [96]:
df_new_features_test.head()

,r1_ability_level,r1_max_ability,r1_ability_rate,r1_max_hero_hit_value,r1_gold_rate,r1_xp_rate,r1_dn_rate,r1_lh_rate,r1_item_uses,r1_item_uses_rate,...,d5_xp_rate,d5_dn_rate,d5_lh_rate,d5_item_uses,d5_item_uses_rate,d5_ability_uses,d5_damage_total,d5_damage_taken_total,d5_damage_inf_rec_total,d5_killed_total
match_id_hash,,,,,,,,,,,,,,,,,,,,,
30cc2d778dca82f2edb568ce9b585caa,0,0,0.000000,60,0.000000,0.000000,0.000000,0.000000,5,5,...,0.000000,0.000000,0.000000,2,2,1,185,187,187,0
70e5ba30f367cea48793b9003fab9d38,6,5035,5.004970,284,5.478431,5.277451,0.000980,0.025490,39,39,...,5.001961,0.001961,0.004902,24,24,45,12491,7863,6343,460
4d9ef74d3a2025d79e9423105fd73d41,7,5298,5.123791,62,4.012963,3.541667,0.000926,0.014815,37,37,...,6.038889,0.011111,0.067593,27,27,6,30441,10133,7928,5876
2bb79e0c1eaac1608e5a09c8e0c6a555,3,5184,8.727273,137,3.415000,3.610000,0.003333,0.035000,5,5,...,6.070000,0.001667,0.041667,8,8,29,16860,9002,3791,1807
bec17f099b01d67edc82dfb5ce735a43,9,5179,3.733958,77,4.120667,4.714000,0.000000,0.017333,28,28,...,8.280000,0.006667,0.085333,115,115,36,77162,23133,10324,24775


In [98]:
train_df_new = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                             'train_feats_new.csv'), 
                                    index_col='match_id_hash')
test_df_new = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                             'test_feats_new.csv'), 
                                    index_col='match_id_hash')

train_df_new_plus = pd.merge(train_df_new, df_new_features, how='left', on = 'match_id_hash')
test_df_new_plus = pd.merge(test_df_new, df_new_features_test, how='left', on = 'match_id_hash')
train_df_new_plus.to_csv('train_feats_new_plus.csv')
test_df_new_plus.to_csv('test_feats_new_plus.csv')

In [100]:
train_df_new_plus.isna()

,game_time,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,...,d5_xp_rate,d5_dn_rate,d5_lh_rate,d5_item_uses_y,d5_item_uses_rate,d5_ability_uses_y,d5_damage_total,d5_damage_taken_total,d5_damage_inf_rec_total,d5_killed_total
match_id_hash,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
b9c57c450ce74a2af79c9ce96fac144d,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6db558535151ea18ca70a6892197db41,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
46a0ddce8f7ed2a8d9bd5edcbb925682,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
b1b35ff97723d9b7ade1c9c3cf48f770,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19c39fe2af2b547e48708ca005c6ae74,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
c96d629dc0c39f0c616d1949938a6ba6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
156c88bff4e9c4668b0f53df3d870f1b,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
df50f738c454d430a33b2bf90cb9ef1a,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [231]:
# copy the dataframe with features
train_df_ex = train_df.copy()

# add new features
add_new_features(train_df_ex, 
                 os.path.join(PATH_TO_DATA, 
                              'train_matches.jsonl'))

In [232]:
test_df_ex = test_df.copy()

# add new features
add_new_features(test_df_ex, 
                 os.path.join(PATH_TO_DATA, 
                              'test_matches.jsonl'))

In [233]:
idx_split = train_df_ex.shape[0]
full_df = pd.concat((train_df_ex, test_df_ex))

In [234]:
train_df_ex.shape, test_df_ex.shape, full_df.shape

((39675, 248), (10000, 248), (49675, 248))

In [235]:
cat_flds = [n for n in full_df.columns if full_df[n].nunique()<20]
cat_flds

['game_mode',
 'lobby_type',
 'r1_camps_stacked',
 'r1_firstblood_claimed',
 'r1_towers_killed',
 'r1_roshans_killed',
 'r2_camps_stacked',
 'r2_firstblood_claimed',
 'r2_towers_killed',
 'r2_roshans_killed',
 'r3_camps_stacked',
 'r3_firstblood_claimed',
 'r3_towers_killed',
 'r3_roshans_killed',
 'r4_camps_stacked',
 'r4_firstblood_claimed',
 'r4_towers_killed',
 'r4_roshans_killed',
 'r5_camps_stacked',
 'r5_firstblood_claimed',
 'r5_towers_killed',
 'r5_roshans_killed',
 'd1_firstblood_claimed',
 'd1_towers_killed',
 'd1_roshans_killed',
 'd2_camps_stacked',
 'd2_firstblood_claimed',
 'd2_towers_killed',
 'd2_roshans_killed',
 'd3_camps_stacked',
 'd3_firstblood_claimed',
 'd3_towers_killed',
 'd3_roshans_killed',
 'd4_camps_stacked',
 'd4_firstblood_claimed',
 'd4_towers_killed',
 'd4_roshans_killed',
 'd5_firstblood_claimed',
 'd5_towers_killed',
 'd5_roshans_killed',
 'radiant_tower_kills',
 'dire_tower_kills']

In [236]:

r_heroes = ['r%s_hero_id' %i for i in range(1,6)]
d_heroes = ['d%s_hero_id' %i for i in range(1,6)]
r_firstblood = ['r%s_firstblood_claimed' %i for i in range(1,6)] 
d_firstblood = ['d%s_firstblood_claimed' %i for i in range(1,6)] 

cat_vars = r_heroes+d_heroes+r_firstblood+d_firstblood+['lobby_type', 'game_mode']
cat_vars

['r1_hero_id',
 'r2_hero_id',
 'r3_hero_id',
 'r4_hero_id',
 'r5_hero_id',
 'd1_hero_id',
 'd2_hero_id',
 'd3_hero_id',
 'd4_hero_id',
 'd5_hero_id',
 'r1_firstblood_claimed',
 'r2_firstblood_claimed',
 'r3_firstblood_claimed',
 'r4_firstblood_claimed',
 'r5_firstblood_claimed',
 'd1_firstblood_claimed',
 'd2_firstblood_claimed',
 'd3_firstblood_claimed',
 'd4_firstblood_claimed',
 'd5_firstblood_claimed',
 'lobby_type',
 'game_mode']

In [237]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49675 entries, a400b8f29dece5f4d266f49f1ae2e98a to 1bbaf6d71e197ebf96c6afa3c921000a
Columns: 248 entries, game_time to diff_tower_kills
dtypes: float64(33), int64(215)
memory usage: 94.4+ MB


# Training 

In [238]:
#from sklearn.model_selection import train_test_split
#X_train_part, X_valid, y_train_part, y_valid = train_test_split(full_df.iloc[:idx_split,:], y_train, test_size = 0.3, random_state=0)

In [239]:
from sklearn.model_selection import ShuffleSplit, cross_val_score
cv = ShuffleSplit(n_splits=5, random_state=1)

In [240]:
#ctb = CatBoostClassifier(iterations = 2000, random_state=1, verbose=False, task_type='GPU', learning_rate=0.1, eval_metric='AUC', 
 #                        cat_features=cat_vars)
#ctb.fit(X_train_part, y_train_part, eval_set=(X_valid, y_valid), plot=True)

In [241]:
#ctb = CatBoostClassifier(random_state=1, verbose=True, task_type='GPU', learning_rate=0.05, eval_metric='AUC', 
#                         cat_features=full_df.select_dtypes(include='object').columns)
#cv_score = cross_val_score(ctb, full_df.iloc[:idx_split,:], y_train, cv=cv, scoring='roc_auc')

In [ ]:
ctb = CatBoostClassifier(iterations = 3000, random_state=1, verbose=True, task_type='GPU', learning_rate=0.045, eval_metric='AUC', 
                         cat_features=cat_vars)
ctb.fit(full_df.iloc[:idx_split,:], y_train)

0:	learn: 0.7286308	total: 91.5ms	remaining: 4m 34s
1:	learn: 0.7393739	total: 178ms	remaining: 4m 26s
2:	learn: 0.7453803	total: 261ms	remaining: 4m 20s
3:	learn: 0.7475493	total: 346ms	remaining: 4m 18s
4:	learn: 0.7495371	total: 423ms	remaining: 4m 13s
5:	learn: 0.7516709	total: 499ms	remaining: 4m 8s
6:	learn: 0.7528787	total: 578ms	remaining: 4m 7s
7:	learn: 0.7536886	total: 655ms	remaining: 4m 4s
8:	learn: 0.7544588	total: 731ms	remaining: 4m 3s
9:	learn: 0.7566331	total: 809ms	remaining: 4m 1s
10:	learn: 0.7564345	total: 882ms	remaining: 3m 59s
11:	learn: 0.7574905	total: 957ms	remaining: 3m 58s
12:	learn: 0.7580614	total: 1.04s	remaining: 3m 58s
13:	learn: 0.7584205	total: 1.12s	remaining: 3m 58s
14:	learn: 0.7589235	total: 1.19s	remaining: 3m 57s
15:	learn: 0.7590345	total: 1.27s	remaining: 3m 56s
16:	learn: 0.7606494	total: 1.34s	remaining: 3m 55s
17:	learn: 0.7619649	total: 1.42s	remaining: 3m 55s
18:	learn: 0.7625357	total: 1.49s	remaining: 3m 54s
19:	learn: 0.7639019	total

159:	learn: 0.8053463	total: 12.3s	remaining: 3m 38s
160:	learn: 0.8054541	total: 12.4s	remaining: 3m 38s
161:	learn: 0.8055461	total: 12.5s	remaining: 3m 38s
162:	learn: 0.8056490	total: 12.5s	remaining: 3m 38s
163:	learn: 0.8057046	total: 12.6s	remaining: 3m 38s
164:	learn: 0.8058429	total: 12.7s	remaining: 3m 38s
165:	learn: 0.8059430	total: 12.8s	remaining: 3m 38s
166:	learn: 0.8059781	total: 12.9s	remaining: 3m 38s
167:	learn: 0.8060242	total: 12.9s	remaining: 3m 37s
168:	learn: 0.8061405	total: 13s	remaining: 3m 37s
169:	learn: 0.8062858	total: 13.1s	remaining: 3m 37s
170:	learn: 0.8064166	total: 13.2s	remaining: 3m 37s
171:	learn: 0.8065027	total: 13.2s	remaining: 3m 37s
172:	learn: 0.8066364	total: 13.3s	remaining: 3m 37s
173:	learn: 0.8068039	total: 13.4s	remaining: 3m 37s
174:	learn: 0.8069090	total: 13.5s	remaining: 3m 37s
175:	learn: 0.8070514	total: 13.5s	remaining: 3m 37s
176:	learn: 0.8071795	total: 13.6s	remaining: 3m 37s
177:	learn: 0.8072463	total: 13.7s	remaining: 3m

315:	learn: 0.8224109	total: 24.2s	remaining: 3m 25s
316:	learn: 0.8224832	total: 24.3s	remaining: 3m 25s
317:	learn: 0.8225622	total: 24.3s	remaining: 3m 25s
318:	learn: 0.8226451	total: 24.4s	remaining: 3m 25s
319:	learn: 0.8227318	total: 24.5s	remaining: 3m 25s
320:	learn: 0.8228405	total: 24.6s	remaining: 3m 24s
321:	learn: 0.8229219	total: 24.6s	remaining: 3m 24s
322:	learn: 0.8230219	total: 24.7s	remaining: 3m 24s
323:	learn: 0.8231025	total: 24.8s	remaining: 3m 24s
324:	learn: 0.8231511	total: 24.9s	remaining: 3m 24s
325:	learn: 0.8232119	total: 24.9s	remaining: 3m 24s
326:	learn: 0.8232798	total: 25s	remaining: 3m 24s
327:	learn: 0.8233843	total: 25.1s	remaining: 3m 24s
328:	learn: 0.8234481	total: 25.2s	remaining: 3m 24s
329:	learn: 0.8235159	total: 25.2s	remaining: 3m 24s
330:	learn: 0.8236616	total: 25.3s	remaining: 3m 24s
331:	learn: 0.8237729	total: 25.4s	remaining: 3m 24s
332:	learn: 0.8238738	total: 25.5s	remaining: 3m 23s
333:	learn: 0.8239837	total: 25.5s	remaining: 3m

471:	learn: 0.8352158	total: 36s	remaining: 3m 13s
472:	learn: 0.8352420	total: 36.1s	remaining: 3m 12s
473:	learn: 0.8353220	total: 36.2s	remaining: 3m 12s
474:	learn: 0.8353838	total: 36.3s	remaining: 3m 12s
475:	learn: 0.8354025	total: 36.3s	remaining: 3m 12s
476:	learn: 0.8355183	total: 36.4s	remaining: 3m 12s
477:	learn: 0.8355790	total: 36.5s	remaining: 3m 12s
478:	learn: 0.8356746	total: 36.6s	remaining: 3m 12s
479:	learn: 0.8357322	total: 36.6s	remaining: 3m 12s
480:	learn: 0.8359063	total: 36.7s	remaining: 3m 12s
481:	learn: 0.8359657	total: 36.8s	remaining: 3m 12s
482:	learn: 0.8360366	total: 36.9s	remaining: 3m 12s
483:	learn: 0.8360817	total: 37s	remaining: 3m 12s
484:	learn: 0.8361189	total: 37s	remaining: 3m 12s
485:	learn: 0.8362025	total: 37.1s	remaining: 3m 12s
486:	learn: 0.8362824	total: 37.2s	remaining: 3m 11s
487:	learn: 0.8363371	total: 37.3s	remaining: 3m 11s
488:	learn: 0.8363972	total: 37.4s	remaining: 3m 11s
489:	learn: 0.8364772	total: 37.4s	remaining: 3m 11s

630:	learn: 0.8454210	total: 48.4s	remaining: 3m 1s
631:	learn: 0.8454291	total: 48.5s	remaining: 3m 1s
632:	learn: 0.8454672	total: 48.6s	remaining: 3m 1s
633:	learn: 0.8454831	total: 48.6s	remaining: 3m 1s
634:	learn: 0.8455466	total: 48.7s	remaining: 3m 1s
635:	learn: 0.8455786	total: 48.8s	remaining: 3m 1s
636:	learn: 0.8456187	total: 48.9s	remaining: 3m 1s
637:	learn: 0.8456583	total: 48.9s	remaining: 3m 1s
638:	learn: 0.8458248	total: 49s	remaining: 3m 1s
639:	learn: 0.8458432	total: 49.1s	remaining: 3m
640:	learn: 0.8458747	total: 49.2s	remaining: 3m
641:	learn: 0.8459401	total: 49.2s	remaining: 3m
642:	learn: 0.8460061	total: 49.3s	remaining: 3m
643:	learn: 0.8460335	total: 49.4s	remaining: 3m
644:	learn: 0.8460997	total: 49.5s	remaining: 3m
645:	learn: 0.8461481	total: 49.5s	remaining: 3m
646:	learn: 0.8462002	total: 49.6s	remaining: 3m
647:	learn: 0.8462247	total: 49.7s	remaining: 3m
648:	learn: 0.8463166	total: 49.8s	remaining: 3m
649:	learn: 0.8463559	total: 49.8s	remaining

...

2312:	learn: 0.9001792	total: 2m 56s	remaining: 52.4s
2313:	learn: 0.9001889	total: 2m 56s	remaining: 52.3s
2314:	learn: 0.9002069	total: 2m 56s	remaining: 52.2s
2315:	learn: 0.9002350	total: 2m 56s	remaining: 52.2s
2316:	learn: 0.9002595	total: 2m 56s	remaining: 52.1s
2317:	learn: 0.9002605	total: 2m 56s	remaining: 52s
2318:	learn: 0.9002890	total: 2m 56s	remaining: 51.9s
2319:	learn: 0.9002892	total: 2m 56s	remaining: 51.8s
2320:	learn: 0.9003277	total: 2m 56s	remaining: 51.8s
2321:	learn: 0.9003278	total: 2m 57s	remaining: 51.7s
2322:	learn: 0.9003319	total: 2m 57s	remaining: 51.6s
2323:	learn: 0.9003648	total: 2m 57s	remaining: 51.5s
2324:	learn: 0.9003818	total: 2m 57s	remaining: 51.5s
2325:	learn: 0.9003999	total: 2m 57s	remaining: 51.4s
2326:	learn: 0.9004169	total: 2m 57s	remaining: 51.3s
2327:	learn: 0.9004263	total: 2m 57s	remaining: 51.2s
2328:	learn: 0.9004402	total: 2m 57s	remaining: 51.2s
2329:	learn: 0.9004648	total: 2m 57s	remaining: 51.1s
2330:	learn: 0.9004672	total: 

2468:	learn: 0.9033183	total: 3m 8s	remaining: 40.5s
2469:	learn: 0.9033535	total: 3m 8s	remaining: 40.4s
2470:	learn: 0.9033759	total: 3m 8s	remaining: 40.3s
2471:	learn: 0.9033774	total: 3m 8s	remaining: 40.3s
2472:	learn: 0.9033882	total: 3m 8s	remaining: 40.2s
2473:	learn: 0.9033962	total: 3m 8s	remaining: 40.1s
2474:	learn: 0.9034040	total: 3m 8s	remaining: 40s
2475:	learn: 0.9034337	total: 3m 8s	remaining: 40s
2476:	learn: 0.9034559	total: 3m 8s	remaining: 39.9s
2477:	learn: 0.9034718	total: 3m 8s	remaining: 39.8s
2478:	learn: 0.9034786	total: 3m 9s	remaining: 39.7s
2479:	learn: 0.9034966	total: 3m 9s	remaining: 39.6s
2480:	learn: 0.9034964	total: 3m 9s	remaining: 39.6s
2481:	learn: 0.9035129	total: 3m 9s	remaining: 39.5s
2482:	learn: 0.9035217	total: 3m 9s	remaining: 39.4s
2483:	learn: 0.9035501	total: 3m 9s	remaining: 39.3s
2484:	learn: 0.9035561	total: 3m 9s	remaining: 39.3s
2485:	learn: 0.9035745	total: 3m 9s	remaining: 39.2s
2486:	learn: 0.9035995	total: 3m 9s	remaining: 39.

2621:	learn: 0.9065537	total: 3m 20s	remaining: 28.9s
2622:	learn: 0.9065596	total: 3m 20s	remaining: 28.8s
2623:	learn: 0.9065709	total: 3m 20s	remaining: 28.7s
2624:	learn: 0.9065740	total: 3m 20s	remaining: 28.6s
2625:	learn: 0.9065920	total: 3m 20s	remaining: 28.6s
2626:	learn: 0.9065999	total: 3m 20s	remaining: 28.5s
2627:	learn: 0.9066352	total: 3m 20s	remaining: 28.4s
2628:	learn: 0.9066705	total: 3m 20s	remaining: 28.3s
2629:	learn: 0.9066985	total: 3m 20s	remaining: 28.3s
2630:	learn: 0.9067042	total: 3m 20s	remaining: 28.2s
2631:	learn: 0.9067616	total: 3m 21s	remaining: 28.1s
2632:	learn: 0.9068117	total: 3m 21s	remaining: 28s
2633:	learn: 0.9068176	total: 3m 21s	remaining: 28s
2634:	learn: 0.9068357	total: 3m 21s	remaining: 27.9s
2635:	learn: 0.9068698	total: 3m 21s	remaining: 27.8s
2636:	learn: 0.9068764	total: 3m 21s	remaining: 27.7s
2637:	learn: 0.9068964	total: 3m 21s	remaining: 27.7s
2638:	learn: 0.9069002	total: 3m 21s	remaining: 27.6s
2639:	learn: 0.9069062	total: 3m

2774:	learn: 0.9099873	total: 3m 32s	remaining: 17.2s
2775:	learn: 0.9099962	total: 3m 32s	remaining: 17.1s
2776:	learn: 0.9100046	total: 3m 32s	remaining: 17.1s
2777:	learn: 0.9100139	total: 3m 32s	remaining: 17s
2778:	learn: 0.9100312	total: 3m 32s	remaining: 16.9s
2779:	learn: 0.9100481	total: 3m 32s	remaining: 16.8s
2780:	learn: 0.9100491	total: 3m 32s	remaining: 16.7s
2781:	learn: 0.9100964	total: 3m 32s	remaining: 16.7s
2782:	learn: 0.9101462	total: 3m 32s	remaining: 16.6s
2783:	learn: 0.9101472	total: 3m 32s	remaining: 16.5s
2784:	learn: 0.9101837	total: 3m 32s	remaining: 16.4s
2785:	learn: 0.9102336	total: 3m 33s	remaining: 16.4s
2786:	learn: 0.9102436	total: 3m 33s	remaining: 16.3s
2787:	learn: 0.9102685	total: 3m 33s	remaining: 16.2s
2788:	learn: 0.9102889	total: 3m 33s	remaining: 16.1s
2789:	learn: 0.9103174	total: 3m 33s	remaining: 16.1s
2790:	learn: 0.9103443	total: 3m 33s	remaining: 16s
2791:	learn: 0.9103706	total: 3m 33s	remaining: 15.9s
2792:	learn: 0.9103726	total: 3m

2927:	learn: 0.9129081	total: 3m 43s	remaining: 5.5s
2928:	learn: 0.9129148	total: 3m 43s	remaining: 5.43s
2929:	learn: 0.9129194	total: 3m 43s	remaining: 5.35s
2930:	learn: 0.9129200	total: 3m 44s	remaining: 5.27s
2931:	learn: 0.9129443	total: 3m 44s	remaining: 5.2s
2932:	learn: 0.9129533	total: 3m 44s	remaining: 5.12s
2933:	learn: 0.9129870	total: 3m 44s	remaining: 5.04s
2934:	learn: 0.9130049	total: 3m 44s	remaining: 4.97s
2935:	learn: 0.9130049	total: 3m 44s	remaining: 4.89s
2936:	learn: 0.9130582	total: 3m 44s	remaining: 4.82s
2937:	learn: 0.9130608	total: 3m 44s	remaining: 4.74s
2938:	learn: 0.9130678	total: 3m 44s	remaining: 4.66s
2939:	learn: 0.9130671	total: 3m 44s	remaining: 4.59s
2940:	learn: 0.9131050	total: 3m 44s	remaining: 4.51s
2941:	learn: 0.9131318	total: 3m 44s	remaining: 4.43s
2942:	learn: 0.9131739	total: 3m 44s	remaining: 4.36s
2943:	learn: 0.9131801	total: 3m 45s	remaining: 4.28s
2944:	learn: 0.9132014	total: 3m 45s	remaining: 4.2s
2945:	learn: 0.9132316	total: 3

In [245]:
y_test_pred = ctb.predict_proba(full_df.iloc[idx_split:,:])[:, 1]

df_submission = pd.DataFrame({'radiant_win_prob': y_test_pred}, 
                                 index=full_df.iloc[idx_split:,:].index)
df_submission.to_csv('submission_base_rf.csv')

In [244]:
#old cv_score, cv_score.mean()